In [7]:
from pinecone import Pinecone, ServerlessSpec
from dotenv import dotenv_values

document_types = {
    "DOCUMENTATIONS": {
        "index_name": "technology-docs",
        "data_dir": "../data/tech_docs"
    },
    "BLOG_POSTS": {
        "index_name": "blog-posts",
        "data_dir": "../data/blog_posts"
    },
    "STACK_OVERFLOW_POSTS": {
        "index_name": "so-posts",
        "data_dir": "../data/so_posts"
    },
}

In [8]:
config = dotenv_values("../.env")
pc = Pinecone(api_key=config["PINECONE_API_KEY"])
indexes = pc.list_indexes().names()
print(indexes)

['blog-posts', 'technology-docs', 'so-posts']


In [9]:
# create indexes
for document_type, values in document_types.items():
    if values["index_name"] not in indexes: 
        print(f"Create index for {document_type}")
        pc.create_index(
            name=values["index_name"],
            dimension=1536,
            metric="cosine",
            spec=ServerlessSpec(
                cloud="aws",
                region="us-east-1"
            )
        )

In [25]:
import re

def clean_up_text(content: str) -> str:
    """
    Remove unwanted characters and patterns in text input.

    :param content: Text input.
    
    :return: Cleaned version of original text input.
    """

    # Fix hyphenated words broken by newline
    content = re.sub(r'(\w+)-\n(\w+)', r'\1\2', content)

    # Remove specific unwanted patterns and characters
    unwanted_patterns = [
        "\\n", "  —", "——————————", "—————————", "—————",
        r'\\u[\dA-Fa-f]{4}', r'\uf075', r'\uf0b7'
    ]
    for pattern in unwanted_patterns:
        content = re.sub(pattern, "", content)

    # Fix improperly spaced hyphenated words and normalize whitespace
    content = re.sub(r'(\w)\s*-\s*(\w)', r'\1-\2', content)
    content = re.sub(r'\s+', ' ', content)

    return content

In [27]:
from llama_index.node_parser import SemanticSplitterNodeParser
from llama_index.embeddings import OpenAIEmbedding
from llama_index.ingestion import IngestionPipeline
from llama_index.core import SimpleDirectoryReader
from llama_index.vector_stores import PineconeVectorStore
from pathlib import Path
import glob
import uuid
import os


embed_model = OpenAIEmbedding(api_key=config["OPENAI_KEY"])

# add data
for document_type, values in document_types.items():
    pinecone_index = pc.Index(values["index_name"])
    #pinecone_index.delete(deleteAll="true")
   
    # Initialize VectorStore
    vector_store = PineconeVectorStore(pinecone_index=pinecone_index)

    cleaned_documents = []

    if not os.path.exists(values["data_dir"]):
        continue

    for dir in glob.glob(values["data_dir"] + "/**"):
        technology_name = dir.split("/")[-1]
        print(technology_name)


        documents = SimpleDirectoryReader(dir).load_data()

        # clean up documents and add technology name to metadata
        for d in documents: 
            cleaned_text = clean_up_text(d.text)
            d.text = cleaned_text
            d.metadata["technology"] = technology_name
            cleaned_documents.append(d)

    # define the ingestion pipeline
    pipeline = IngestionPipeline(
    transformations=[
        SemanticSplitterNodeParser(
            buffer_size=1,
            breakpoint_percentile_threshold=95, 
            embed_model=embed_model,
            ),
        embed_model,
        ],
        vector_store=vector_store
    )

    # run the ingestion pipeline
    pipeline.run(documents=cleaned_documents)

    

docker
[Document(id_='be8da2d4-99c8-4e02-864a-e7ab58498717', embedding=None, metadata={'page_label': '1', 'file_name': 'dockerfile.pdf', 'file_path': '/home/simisimon/GitHub/cval/src/../data/tech_docs/docker/dockerfile.pdf', 'file_type': 'application/pdf', 'file_size': 110692, 'creation_date': '2024-05-29', 'last_modified_date': '2024-05-29', 'technology': 'docker'}, excluded_embed_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], excluded_llm_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], relationships={}, text='Dockerfile referenceTable of contents\x7fOverview\x7fFormat\x7fParser directives\x7fsyntax\x7fescape\x7fEnvironment replacement\x7f.dockerignore file\x7fShell and exec form\x7fExec form\x7fShell form\x7fUse a different shell\x7fFROM\x7fUnderstand how ARG and FROM interact\x7fRUN\x7fCache invalidation for RUN instructions\x7fRUN --mount\x7fRUN -